In [ ]:
from gym import Env
import gym
from gym.spaces import Discrete, Box
import numpy as np
import deepchem as dc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import torch
from pysmiles import read_smiles
import networkx as nx
import dgllife
from rdkit import Chem
import random
import spektral 
import dgl.nn.pytorch as dglnn
import torch.nn as nn
import dgl
from dgl import DGLGraph
from torch.nn.utils.rnn import pad_sequence



import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from torch.distributions import Categorical
import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter

%run MoleculeGenerator2.ipynb
%run Discrim.ipynb
%run Rewards.ipynb
%run PPO_WITH_TRICKS.ipynb
%run ISO_ENV.ipynb
%run DiscTraining.ipynb


In [ ]:
max_nodes = 12
num_atom_types = 14
num_node_feats = 34
num_edge_types = 3
bond_padding = 12

batch_size = 32
timesteps_per_batch = 1200
max_timesteps_per_episode = 30
clip = 0.08
a_lr = 1e-4
c_lr = 3e-4
n_updates_per_iteration = 6
gamma = .95



synthReward = Synthesizability()
sizeReward = SizeReward()
writer = SummaryWriter()


rewardModel = FinalRewardModule(writer,[synthReward,sizeReward])

env = GraphGenEnv(max_nodes,num_atom_types,num_node_feats,num_edge_types,bond_padding,rewardModel)



testEnv = PPO_MAIN(env,batch_size,timesteps_per_batch,clip,a_lr,c_lr,n_updates_per_iteration,max_timesteps_per_episode,gamma)





In [ ]:
path = '...'
in_dim = num_node_feats
hidden_dim = 300
lr = 3e-4

In [ ]:
discReward = DiscReward(path,in_dim,hidden_dim,lr)



In [ ]:
PPO_agent = testEnv
Disc = discReward
epochs = 20
G_steps = 100
D_steps = 100
K = 1
G_pretrain_steps = 100
D_train_size = 256
D_batch_size = 32
smiles_values = smiles_values
rewardModel = FinalRewardModule(writer,[synthReward,sizeReward,discReward])

In [ ]:
adversarialTraining = AdversarialTraining(testEnv,discReward,epochs,
                                          G_steps, D_steps,K,G_pretrain_steps,
                                          D_train_size, D_batch_size, smiles_values)

In [ ]:
adversarialTraining.train(epochs=10)

In [ ]:
testEnv.learn(100000)